In [1]:
# CNN algorithm implementation was used from https://www.kaggle.com/code/huseyinndogan/cnn-tumor-classification-with-96-accuracy and adapted a bit
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [11]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow


In [9]:
import cv2


ModuleNotFoundError: No module named 'cv2'

In [2]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.initializers import HeUniform, GlorotUniform
from sklearn.preprocessing import StandardScaler
import numpy as npa
import matplotlib.pyplot as plt
import pandas as pd
import os
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix , accuracy_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import GlobalAveragePooling2D, LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import losses
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator



ModuleNotFoundError: No module named 'tensorflow'

### 📌 1. Path Identification

In [21]:
import pathlib
data_dir = pathlib.Path(r"/NIC-project/brain-tumor-mri-dataset")

train_dir = 'brain-tumor-mri-dataset/Training'
test_dir = 'brain-tumor-mri-dataset/Testing'


### 📌 2. Data Preprocessing with ImageDataGenerator

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

### 📌 3. Defining Train & Test Folders

In [22]:
train_dir = str(data_dir / "Training") 
test_dir = str(data_dir / "Testing")    

### 📌 4. Define Image Size & Batch Size for Model

In [15]:
batch_size = 32
img_height = 240
img_width = 240

### 📌 5. Loading Training and Test Datasets

In [23]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (img_height , img_width),
    batch_size= batch_size,
    class_mode = "categorical"
    )

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size = (img_height , img_width),
    batch_size = batch_size,
    class_mode = "categorical"

    )

print(r"Class Label :" , train_generator.class_indices)


Found 5712 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.
Class Label : {'glioma': 0, 'meningioma': 1, 'notumor': 2, 'pituitary': 3}


### DATA PREPROCESSING SUMMARY

🚀 What Does the Code Do in Brief?

* Determines the dataset folder (data_dir).

* Creates an ImageDataGenerator to rescale the images.

* Determines the training (train_dir) and test (test_dir) data paths.

* Resize the images to 240x240 and load them with flow_from_directory().

* Prints the class labels.

### 📌 6.LET'S DEVELOP THE MODEL

In [24]:

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,          
    width_shift_range=0.2,     
    height_shift_range=0.2,    
    shear_range=0.2,           
    zoom_range=0.2,             
    horizontal_flip=True,       
    fill_mode='nearest'       
)

test_datagen = ImageDataGenerator(rescale=1./255)


In [25]:
model = Sequential([
    Conv2D(32 , (5,5) , activation="relu" ,padding="valid", kernel_initializer=HeUniform()),
    MaxPooling2D(2,2),
    BatchNormalization(),

    Conv2D(64 , (5,5) , activation="relu" ,padding="valid", kernel_initializer=HeUniform()),
    MaxPooling2D(2,2),
    BatchNormalization(),

    Conv2D(128 , (5,5) , activation="relu" ,padding="valid", kernel_initializer=HeUniform()),
    MaxPooling2D(2,2),
    BatchNormalization(),

    Flatten(),

    Dense(64 , activation="relu" , kernel_initializer=HeUniform()),
    Dropout(0.2),
    BatchNormalization(),
    Dense(128 , activation="relu" , kernel_initializer=HeUniform()),
    Dropout(0.2),
    BatchNormalization(),
    Dense(256 , activation="relu" , kernel_initializer=HeUniform()),
    Dropout(0.3),
    BatchNormalization(),

    Dense(4 , activation="softmax" , kernel_initializer=GlorotUniform()),

    ])

In [26]:
model.compile(optimizer=Adam(learning_rate=0.0001) , loss="categorical_crossentropy", metrics=["accuracy"])

In [28]:
early_stop = EarlyStopping(patience=3,monitor="val_loss", restore_best_weights=True)

fit = model.fit(train_generator,epochs=10 #(epoch count is reduced due to process length.)
, validation_data=(test_generator) , callbacks=[early_stop])

Epoch 1/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 534s 3s/step - accuracy: 0.6769 - loss: 0.8260 - val_accuracy: 0.7292 - val_loss: 0.7150
Epoch 2/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 310s 2s/step - accuracy: 0.7055 - loss: 0.7714 - val_accuracy: 0.6903 - val_loss: 0.8881
Epoch 3/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 321s 2s/step - accuracy: 0.7057 - loss: 0.7659 - val_accuracy: 0.6728 - val_loss: 0.8499
Epoch 4/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 464s 3s/step - accuracy: 0.7264 - loss: 0.7142 - val_accuracy: 0.7178 - val_loss: 0.8003


In [30]:
model.save("brain_tumor_model.keras")


In [31]:
model = tf.keras.models.load_model("brain_tumor_model.keras")


C:\Users\hamza\PycharmProjects\NN_project\.venv\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 28 variables whereas the saved optimizer has 54 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
### LET'S TEST ON INPUT IMAGE 

In [36]:
def predict_image(image_path, model):
    img = cv2.imread(image_path)  # Image loading
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Change image color to rgb
    img = cv2.resize(img, (240, 240))  # Change the size of image to fit the model
    img = img / 255.0  # Normalise
    img = npa.expand_dims(img, axis=0) 

    prediction = model.predict(img)  # Predict image 
    predicted_class = npa.argmax(prediction)  # Get predicted image class - from 0 to 3

    class_labels = {0: "Glioma", 1: "Meningioma", 2: "No Tumor", 3: "Pituitary Tumor"}
    return class_labels[predicted_class]  # Get the name of LABEL 

image_path = "brain-tumor-mri-dataset/test-glioma.jpg"  
result = predict_image(image_path, model) # func call
print(f"Предсказанный класс: {result}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
Предсказанный класс: Glioma


# LET'S EVALUATE THE MODEL

In [15]:

x_test, y_test = next(test_generator)  


predictions = model.predict(x_test)


predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(y_test, axis=1)


print("Accuracy:", accuracy_score(true_classes, predicted_classes))
print("Classification Report:")
print(classification_report(true_classes, predicted_classes))
print("Confusion Matrix:")
print(confusion_matrix(true_classes, predicted_classes))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 487ms/step
Accuracy: 0.90625
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.71      0.83         7
           1       0.71      0.83      0.77         6
           2       0.91      1.00      0.95        10
           3       1.00      1.00      1.00         9

    accuracy                           0.91        32
   macro avg       0.91      0.89      0.89        32
weighted avg       0.92      0.91      0.91        32

Confusion Matrix:
[[ 5  2  0  0]
 [ 0  5  1  0]
 [ 0  0 10  0]
 [ 0  0  0  9]]


* If you get an error, you can check the code below.

In [16]:

'''

all_predictions = []
all_true_classes = []


for x_batch, y_batch in test_generator:
    predictions = model.predict(x_batch)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(y_batch, axis=1)
    

    all_predictions.extend(predicted_classes)
    all_true_classes.extend(true_classes)


print("Accuracy:", accuracy_score(all_true_classes, all_predictions))
print("Classification Report:")

print(classification_report(all_true_classes, all_predictions))

print("Confusion Matrix:")
print(confusion_matrix(all_true_classes, all_predictions))


'''



'\n\nall_predictions = []\nall_true_classes = []\n\n\nfor x_batch, y_batch in test_generator:\n    predictions = model.predict(x_batch)\n    predicted_classes = np.argmax(predictions, axis=1)\n    true_classes = np.argmax(y_batch, axis=1)\n    \n\n    all_predictions.extend(predicted_classes)\n    all_true_classes.extend(true_classes)\n\n\nprint("Accuracy:", accuracy_score(all_true_classes, all_predictions))\nprint("Classification Report:")\n\nprint(classification_report(all_true_classes, all_predictions))\n\nprint("Confusion Matrix:")\nprint(confusion_matrix(all_true_classes, all_predictions))\n\n\n'

### I hope it was useful for you.